In [17]:
from metadsl import *

import dataclasses

We can make a natural number type:

In [16]:
@dataclasses.dataclass(frozen=True)
class NaturalNumber(Instance):

    def __add__(self, other: "NaturalNumber") -> "NaturalNumber":
        return NaturalNumber(Call(NaturalNumber.__add__, (self, other)))

And an expression about it:

In [14]:
n = NaturalNumber(123) + NaturalNumber(456) + NaturalNumber(456)
n + n

NaturalNumber(__value__=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(NaturalNumber(__value__=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(NaturalNumber(__value__=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(NaturalNumber(__value__=123), NaturalNumber(__value__=456)))), NaturalNumber(__value__=456)))), NaturalNumber(__value__=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(NaturalNumber(__value__=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(NaturalNumber(__value__=123), NaturalNumber(__value__=456)))), NaturalNumber(__value__=456)))))))

In [15]:
dataclasses.astuple(n + n)

((<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
  (((<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
     (((<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
        ((123,), (456,))),),
      (456,))),),
   ((<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
     (((<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
        ((123,), (456,))),),
      (456,))),))),)

Now we can instatiate the expression:

In [9]:
expr = Expression.from_instance(n)
expr

Expression(type=<class '__main__.NaturalNumber'>, value=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(Expression(type=<class '__main__.NaturalNumber'>, value=Call(function=<function NaturalNumber.__add__ at 0x10641a6a8>, args=(Expression(type=<class '__main__.NaturalNumber'>, value=123), Expression(type=<class '__main__.NaturalNumber'>, value=456)))), Expression(type=<class '__main__.NaturalNumber'>, value=456))))

In [10]:
dataclasses.astuple(expr)

(__main__.NaturalNumber,
 (<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
  ((__main__.NaturalNumber,
    (<function __main__.NaturalNumber.__add__(self, other: 'NaturalNumber') -> 'NaturalNumber'>,
     ((__main__.NaturalNumber, 123), (__main__.NaturalNumber, 456)))),
   (__main__.NaturalNumber, 456))))

In [12]:
expr.instance == n

True